# Semantic Search
Semantic search is [defined as "search with meaning."](https://en.wikipedia.org/wiki/Semantic_search) It is key for effective knowledge retrieval.

As opposed to traditional, lexical, search which finds matches based on keywords, semantic search seeks to improve search quality and accuracy by understanding search intent and pulling results that match the user’s contextual meaning.  Semantic search is often used in reference to text embedding and vector similarity search, but this is just one implementation aspect of it. Knowledge graph and symbolic query logic can also play a critical role in making semantic search a reality. 

If all you care about is analyzing a set of documents on a file system, then sure, vector indexing and search may be sufficient.  However, once you need to retrieve and make inferences about people, places, and things connected to those documents, Knowledge graph becomes key. 

If documents are the entities of interest, for example: "find all documents that talk about pharma related things," then text embeddings with vector similarity search suffices.  But, what if we want second or third-order entities related to the documents?  For example, "find investors who are most focused on pharma related strategies," how would we efficiently search for them at scale in an enterprise setting?

This is what we demonstrate below.  We will also show how you can use graph relationships and Graph Data Science algorithms to further improve search results, especially in common scenarios where the presence of text data is inconsistent or sparse.

## Connect to Neo4j

In [ ]:
# username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match your credentials
NEO4J_URI = 'neo4j+s://6688b25b.databases.neo4j.io'
NEO4J_PASSWORD = '_kogrNk53u8oTk5be55kmit1kHGdhZj98yJlG-VYSR'

In [4]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')

## Neo4j Vector Index
We will need to create a vector index for similarity search on Document nodes. Neo4j offers a vector index that enables Approximate Nearest Neighbor Search (ANN). Let's creat an index.

In [ ]:
gds.run_cypher("CALL db.index.vector.createNodeIndex('document-embeddings', 'Document', 'textEmbedding', 768, 'cosine')")

You can see that vector index has been created using `SHOW INDEXES`

In [8]:
gds.run_cypher(''' 
SHOW INDEXES YIELD name, type, labelsOrTypes, properties, options
WHERE type = "VECTOR"
''')

,name,type,labelsOrTypes,properties,options
0,document-embeddings,VECTOR,[Document],[textEmbedding],"{'indexProvider': 'vector-1.0', 'indexConfig':..."


## Deep Semantic Search with Knowledge Graph
Now that we have an index.  Let’s use it in action.
In this case, we will Answer The Question - "What are the companies associated with energy, oil and gas?". 
Remember we do not have documents on investment managers, just companies, and there can be multiple documents for each company.

## Dense Retrieval using Neo4j

In [5]:
from langchain.docstore.document import Document
from typing import (
    List,
    Tuple,
)
import json

def to_df(results: List[Tuple[Document, float]]):
    return pd.DataFrame({
        "doc_id": [r[0].metadata.get('documentId') for r in results],
        "score": [r[1] for r in results],
        "text": [r[0].page_content for r in results]
    })

In [39]:
from langchain.embeddings.vertexai import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Neo4jVector
from langchain.vectorstores.neo4j_vector import SearchType
import pandas as pd

db = Neo4jVector.from_existing_index(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    embedding=VertexAIEmbeddings(),
    index_name='document-embeddings',
    keyword_index_name='doc_text',
    search_type=SearchType.VECTOR
)
query = 'What are the companies associated with energy, oil and gas?'
results = db.similarity_search_with_score(query, k=10)
res_df = to_df(results)
res_df

,doc_id,score,text
0,"Goldman Sachs BDC, Inc.34",0.846822,Conergy Asia & ME Pte. LTD.\n\n\nConstruction ...
1,BERKSHIRE HATHAWAY INC25,0.845432,The Great Britain distribution companies consi...
2,BERKSHIRE HATHAWAY INC24,0.842114,MidAmerican Energy Company (“MEC”) is a regula...
3,BERKSHIRE HATHAWAY INC23,0.840685,Berkshire currently has a 91.1% ownership inte...
4,ROPER TECHNOLOGIES I5,0.838911,"FTI\n - provides flow meter calibrators, and c..."
5,BERKSHIRE HATHAWAY INC34,0.838776,BHE and its energy subsidiaries continue to fo...
6,BERKSHIRE HATHAWAY INC40,0.838742,Lubrizol \n\n\nThe Lubrizol Corporation (“Lubr...
7,"STANLEY BLACK & DECKER, INC7",0.837772,The Engineered Fastening business primarily se...
8,CANADIAN PACIFIC KANSAS CITY LTD16,0.836339,Forest products traffic includes pulp and pape...
9,BERKSHIRE HATHAWAY INC46,0.836064,Transportation Products\n serves the automotiv...


Lets take a look at the first result and then understand whether the chunk has the information we asked for.

In [40]:
res_df['text'][0]

'Conergy Asia & ME Pte. LTD.\n\n\nConstruction & Engineering\n\n\n\xa0\n\n\n01/11/21\n\n\n\xa0\n\n\n3,126,780\n\n\n\xa0\n\n\n\xa0\n\n\n5,300\n\n\n\xa0\n\n\n\xa0\n\n\n—\n\n\n\xa0\n\n\n\xa0^ (1) (3) (6)\n\n\n\n\n\n\nCountry Fresh Holding Company Inc.\n\n\nFood Products\n\n\n\xa0\n\n\n04/29/19\n\n\n\xa0\n\n\n1,514\n\n\n\xa0\n\n\n\xa0\n\n\n888\n\n\n\xa0\n\n\n\xa0\n\n\n—\n\n\n\xa0\n\n\n\xa0(2) (3) (6)\n\n\n\n\n\n\nElah Holdings, Inc.\n\n\nCapital Markets\n\n\n\xa0\n\n\n05/09/18\n\n\n\xa0\n\n\n111,650\n\n\n\xa0\n\n\n\xa0\n\n\n5,238\n\n\n\xa0\n\n\n\xa0\n\n\n5,396\n\n\n\xa0\n\n\n\xa0^ (2) (3) (6)\n\n\n\n\n\n\nExostar LLC - Class B\n\n\nAerospace & Defense\n\n\n\xa0\n\n\n07/06/20\n\n\n\xa0\n\n\n31,407\n\n\n\xa0\n\n\n\xa0\n\n\n—\n\n\n\xa0\n\n\n\xa0\n\n\n25\n\n\n\xa0\n\n\n\xa0(2) (3) (6)\n\n\n\n\n\n\nFoundation Software - Class B\n\n\nConstruction & Engineering\n\n\n\xa0\n\n\n08/31/20\n\n\n\xa0\n\n\n11,826\n\n\n\xa0\n\n\n\xa0\n\n\n—\n\n\n\xa0\n\n\n\xa0\n\n\n9\n\n\n\xa0\n\n\n\xa0(2) (3) (6)\n\n\n\

## Re-rank results
If you explore the results, you may find some irrelevant results. The retriever has to be efficient for large document collections with millions of entries. However, it might return irrelevant candidates.
A re-ranker based on a Cross-Encoder can substantially improve the final results for the user. The query and a possible document is passed simultaneously to transformer network, which then outputs a single score between 0 and 1 indicating how relevant the document is for the given query.

We will use a Cross-encoder model named `ms-marco-MiniLM-L-6-v2` from SentenceTransformers library.

In [43]:
def rerank_results(res_df):
    from sentence_transformers import SentenceTransformer, CrossEncoder, util
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    passages = []
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, f"Text: {hit['text']}"] for _,hit in res_df[:].iterrows()]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    res_df['cross-score'] = cross_scores

    # Output of top-10 hits from re-ranker
    print("Top-10 Cross-Encoder Re-ranker hits")
    hits = res_df.sort_values('cross-score', ascending=False).reset_index(drop=True)
    for _,hit in hits.iterrows():
        print("\t{:.3f}\t{}".format(hit['cross-score'], hit['doc_id']))
    return hits

In [44]:
ranked_results = rerank_results(res_df)
ranked_results

Top-10 Cross-Encoder Re-ranker hits
	-2.413	BERKSHIRE HATHAWAY INC23
	-3.615	BERKSHIRE HATHAWAY INC24
	-3.911	Goldman Sachs BDC, Inc.34
	-4.144	CANADIAN PACIFIC KANSAS CITY LTD16
	-4.773	BERKSHIRE HATHAWAY INC25
	-5.857	STANLEY BLACK & DECKER, INC7
	-5.920	BERKSHIRE HATHAWAY INC34
	-7.938	ROPER TECHNOLOGIES I5
	-8.347	BERKSHIRE HATHAWAY INC40
	-9.472	BERKSHIRE HATHAWAY INC46


,doc_id,score,text,cross-score
0,BERKSHIRE HATHAWAY INC23,0.840685,Berkshire currently has a 91.1% ownership inte...,-2.412767
1,BERKSHIRE HATHAWAY INC24,0.842114,MidAmerican Energy Company (“MEC”) is a regula...,-3.615500
2,"Goldman Sachs BDC, Inc.34",0.846822,Conergy Asia & ME Pte. LTD.\n\n\nConstruction ...,-3.911106
3,CANADIAN PACIFIC KANSAS CITY LTD16,0.836339,Forest products traffic includes pulp and pape...,-4.144452
4,BERKSHIRE HATHAWAY INC25,0.845432,The Great Britain distribution companies consi...,-4.772871
5,"STANLEY BLACK & DECKER, INC7",0.837772,The Engineered Fastening business primarily se...,-5.857329
6,BERKSHIRE HATHAWAY INC34,0.838776,BHE and its energy subsidiaries continue to fo...,-5.920151
7,ROPER TECHNOLOGIES I5,0.838911,"FTI\n - provides flow meter calibrators, and c...",-7.938165
8,BERKSHIRE HATHAWAY INC40,0.838742,Lubrizol \n\n\nThe Lubrizol Corporation (“Lubr...,-8.347017
9,BERKSHIRE HATHAWAY INC46,0.836064,Transportation Products\n serves the automotiv...,-9.472342


As seen above, the cross-encoder finds this passage from Bershire Hathaway to be more relevant to the query

In [42]:
ranked_results['text'][0]

'Berkshire currently has a 91.1% ownership interest in Berkshire Hathaway Energy Company (“BHE”). BHE is a global energy company with subsidiaries and affiliates that generate, transmit, store, distribute and supply energy. BHE’s locally managed businesses are organized as separate operating units. BHE’s domestic regulated energy interests are comprised of four regulated utility companies serving approximately 5.2\xa0million retail customers, five interstate natural gas pipeline companies with approximately 21,100 miles of operated pipeline having a design capacity of approximately 21\xa0billion cubic feet of natural gas per day and ownership interests in electricity transmission businesses. BHE’s Great Britain electricity distribution subsidiaries serve about 3.9\xa0million electricity end-users and its electricity transmission-only business in Alberta, Canada serves approximately 85% of Alberta’s population. BHE’s interests also include a diversified portfolio of independent power pr

## Hybrid Search with Neo4j
Vector Search is not the only solution. Often you will find a hybrid approach - a combination of Vector and Fulltext search perform better. You can do both in Neo4j.

To do a Fulltext Search, lets first create a fulltext index

In [228]:
gds.run_cypher("CREATE FULLTEXT INDEX doc_text IF NOT EXISTS FOR (n:Document) ON EACH [n.text]")

""


In [229]:
gds.run_cypher(''' 
SHOW INDEXES YIELD name, type, labelsOrTypes, properties, options
WHERE type = "FULLTEXT"
''')

,name,type,labelsOrTypes,properties,options
0,doc_text,FULLTEXT,[Document],[text],"{'indexProvider': 'fulltext-1.0', 'indexConfig..."


In [45]:
from langchain.embeddings.vertexai import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Neo4jVector
from langchain.vectorstores.neo4j_vector import SearchType

db = Neo4jVector.from_existing_index(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    embedding=VertexAIEmbeddings(),
    index_name='document-embeddings',
    keyword_index_name='doc_text',
    search_type=SearchType.HYBRID # Hybrid search type
)

results = db.similarity_search_with_score(query, k=10)
res_df = to_df(results)
res_df

,doc_id,score,text
0,"Goldman Sachs BDC, Inc.102",1.000000,—\n\n\n \n\n\n\n\n\n\nTrading Companies & Dist...
1,BERKSHIRE HATHAWAY INC37,0.957910,Investment casting technology involves a multi...
2,Martin Marietta Materia12,0.951432,Cement consumption is dependent on the time of...
3,BERKSHIRE HATHAWAY INC26,0.930401,"Northern Natural, based in Nebraska, operates ..."
4,DELTA AI21,0.892080,Our results of operations are significantly im...
5,CANADIAN PACIFIC KANSAS CITY LTD16,0.890692,Forest products traffic includes pulp and pape...
6,Delta Apparel Inc14,0.859891,"In February 2022, we removed 27 older model ci..."
7,"Goldman Sachs BDC, Inc.34",0.846822,Conergy Asia & ME Pte. LTD.\n\n\nConstruction ...
8,BERKSHIRE HATHAWAY INC25,0.845432,The Great Britain distribution companies consi...
9,BERKSHIRE HATHAWAY INC24,0.842114,MidAmerican Energy Company (“MEC”) is a regula...


Lets re-rank our results for relevancy

In [47]:
ranked_results = rerank_results(res_df)
ranked_results

Top-10 Cross-Encoder Re-ranker hits
	-3.615	BERKSHIRE HATHAWAY INC24
	-3.911	Goldman Sachs BDC, Inc.34
	-4.144	CANADIAN PACIFIC KANSAS CITY LTD16
	-4.773	BERKSHIRE HATHAWAY INC25
	-6.181	Martin Marietta Materia12
	-6.294	Goldman Sachs BDC, Inc.102
	-7.015	BERKSHIRE HATHAWAY INC26
	-7.793	BERKSHIRE HATHAWAY INC37
	-8.990	DELTA AI21
	-10.271	Delta Apparel Inc14


,doc_id,score,text,cross-score
0,BERKSHIRE HATHAWAY INC24,0.842114,MidAmerican Energy Company (“MEC”) is a regula...,-3.615500
1,"Goldman Sachs BDC, Inc.34",0.846822,Conergy Asia & ME Pte. LTD.\n\n\nConstruction ...,-3.911106
2,CANADIAN PACIFIC KANSAS CITY LTD16,0.890692,Forest products traffic includes pulp and pape...,-4.144452
3,BERKSHIRE HATHAWAY INC25,0.845432,The Great Britain distribution companies consi...,-4.772871
4,Martin Marietta Materia12,0.951432,Cement consumption is dependent on the time of...,-6.180655
5,"Goldman Sachs BDC, Inc.102",1.000000,—\n\n\n \n\n\n\n\n\n\nTrading Companies & Dist...,-6.294070
6,BERKSHIRE HATHAWAY INC26,0.930401,"Northern Natural, based in Nebraska, operates ...",-7.015031
7,BERKSHIRE HATHAWAY INC37,0.957910,Investment casting technology involves a multi...,-7.792734
8,DELTA AI21,0.892080,Our results of operations are significantly im...,-8.990334
9,Delta Apparel Inc14,0.859891,"In February 2022, we removed 27 older model ci...",-10.270824


In [49]:
ranked_results['text'][4]

'Cement consumption is dependent on the time of year and prevalent weather conditions. According to the Portland Cement Association, nearly two-thirds of U.S. cement consumption occurs in the six months between May and October.\xa0\xa0Approximately 70% to 75% of all cement shipments are sent to ready mixed concrete operators. The rest are shipped to manufacturers of concrete-related products, contractors, materials dealers and oil well/mining/drilling companies, as well as government entities.\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSOAR to a Sustainable Future\n\n\n\n\n\n\n\n\nForm 10-K\xa0\xa0\n♦\n \nPage 3\n\n\n\n\n\n\n\xa0\n\n\n\n\nPart I\xa0\xa0\n♦\n\xa0\xa0Item 1 – Business\n\n\nEnergy accounted for approximately 23% of the cement production cost profile in 2021. Therefore, profitability of cement is affected by changes in energy prices and the available supply of these products.\xa0\xa0The Company currently has fixed-price supply contracts for coal and natural gas, bu

The Hybrid search brought in additional results like `Martin Marietta Material2` which also has content related to energy, oil & gas. The re-ranker helped rank the results. 

## Semantic Search with Multi-hop retrieval using Knowledge Graph

In [50]:
# Semantic query, lets use these key words to search
db = Neo4jVector(
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD,
    embedding = VertexAIEmbeddings()
)

Now let's use that query vector to search for companies.  Remember, companies have multiple documents so we will need to use a graph traversal on top of a document lookup to find which companies are most similar.

In [51]:
query_vector = db.embedding.embed_query(text=query)
query_vector[0:5]

[0.06058839336037636,
 0.0039505185559391975,
 -0.05093487352132797,
 0.01724359206855297,
 0.0011352039873600006]

In [52]:
# Search for similar companies
res_df = db.query("""
CALL db.index.vector.queryNodes('document-embeddings', 10, $queryVector)
YIELD node AS similarDocuments, score

MATCH (similarDocuments)<-[:HAS]-(c:Company)
RETURN c.companyName as companyName, avg(score) AS score
ORDER BY score DESC LIMIT 10
""", params =  {'queryVector': query_vector})
res_df

[{'companyName': 'Goldman Sachs Grp', 'score': 0.8468223214149475},
 {'companyName': 'Berkshire Hathaway Inc B', 'score': 0.8403021693229675},
 {'companyName': 'ROPER TECHNOLOGIES', 'score': 0.838910698890686},
 {'companyName': 'STANLEY BLACK &amp; DECKER', 'score': 0.8377716541290283},
 {'companyName': 'CANADIAN NATIONAL RAILWAY', 'score': 0.8363386392593384}]

The above result is based on the limited set of documents we have in the DB. The scores are based on Average similarity scores across chunks

Now let's take this one step further and find investment managers who are most heavily focused in Energy.  This will involve a bit more Cypher for a 2-hop traversal. 

In [53]:
# Search for managers with significiant investments in area
res_df = gds.run_cypher("""
CALL db.index.vector.queryNodes('document-embeddings', 1000, $queryVector)
YIELD node AS similarDocuments, score
MATCH (similarDocuments)<-[:HAS]-(c:Company)
WITH c, avg(score) AS score ORDER BY score LIMIT 100
MATCH (c)<-[r:OWNS]-(m:Manager)
WITH m, r.value as value, score*r.value as weightedScore
WITH m.managerName AS managerName, sum(weightedScore) AS aggScore, sum(value) AS aggValue
RETURN managerName, aggScore/aggValue AS score ORDER BY score DESC LIMIT 1000

""", params =  {'queryVector': query_vector})
res_df

,managerName,score
0,"BASSETT HARGROVE INVESTMENT COUNSEL, LLC",0.802753
1,Rempart Asset Management Inc.,0.802098
2,TIGER MANAGEMENT L.L.C.,0.798163
3,"Emery Howard Portfolio Management, Inc.",0.797394


And we can see that our top result is `BASSETT HARGROVE INVESTMENT COUNSEL, LLC	`

## Expanding Available Data for Knowledge Retrieval
Not every element in your data will have rich text data, and further, much like we only have 10K documents for some companies, your use cases may also have incomplete, unevenly distributed text data. 

We can check our top result investment manager to this. 

In [54]:
gds.run_cypher('''
MATCH (m:Manager {managerName: $managerName})-[:OWNS]->(c:Company)-[:HAS]->(d:Document)
WITH m, count(DISTINCT c) AS ownedCompaniesWithDocs
MATCH (m:Manager {managerName: $managerName})-[:OWNS]->(c:Company)
RETURN m.managerName AS managerName, ownedCompaniesWithDocs, count(DISTINCT c) AS totalOwnedCompanies
''', params =  {'managerName':'BASSETT HARGROVE INVESTMENT COUNSEL, LLC'})

,managerName,ownedCompaniesWithDocs,totalOwnedCompanies
0,"BASSETT HARGROVE INVESTMENT COUNSEL, LLC",22,51


This manager has significantly more other companies they own without documents.  We can use Graph Data Science Node Similarity to find the managers that have the most overlap to this one which should give us other Energy companies that we missed due to sparse text data.

In [58]:
g, _ = gds.graph.project('proj', ['Company', 'Manager'], {'OWNS':{'properties':['value']}})

In [59]:
gds.nodeSimilarity.write(g, writeRelationshipType='SIMILAR', writeProperty='score', relationshipWeightProperty='value')

preProcessingMillis                                                       0
computeMillis                                                             5
writeMillis                                                              22
postProcessingMillis                                                     -1
nodesCompared                                                             4
relationshipsWritten                                                     10
similarityDistribution    {'min': 6.620865315198898e-05, 'p5': 6.6208653...
configuration             {'writeProperty': 'score', 'writeRelationshipT...
Name: 0, dtype: object

In [60]:
g.drop()

graphName                                                             proj
database                                                             neo4j
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                              247
relationshipCount                                                      172
configuration            {'relationshipProjection': {'OWNS': {'aggregat...
density                                                           0.002831
creationTime                           2023-09-28T10:29:54.708352797+00:00
modificationTime                       2023-09-28T10:29:54.722032139+00:00
schema                   {'graphProperties': {}, 'nodes': {'Manager': {...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'Manager': {...
Name: 0, dtype: object

And now we can pull back other relevant results

In [62]:
gds.run_cypher('''
MATCH (m0:Manager {managerName: $managerName})-[r:SIMILAR]->(m:Manager)
RETURN DISTINCT m.managerName AS managerName, r.score AS score
ORDER BY score DESC LIMIT 10
''', params =  {'managerName':'BASSETT HARGROVE INVESTMENT COUNSEL, LLC'})

,managerName,score
0,Rempart Asset Management Inc.,0.023110
1,"Emery Howard Portfolio Management, Inc.",0.010626
2,TIGER MANAGEMENT L.L.C.,0.000066


## Clean Up

In [56]:
gds.run_cypher('MATCH (M:Manager)-[s:SIMILAR]->() DELETE s')

""
